In [15]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tsfresh as tsf
import sklearn


In [14]:
!pip install matplotlib seaborn 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 5.5 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 6.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 3.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 3.3 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 4.4 MB/s eta 0:00:00


In [54]:
# Import data
X_train = pd.read_csv('../Data/X_train.csv', sep=",")
y_train = pd.read_csv('../Data/y_train.csv', sep=",")
X_test = pd.read_csv('../Data/X_test.csv', sep=",")
y_test = pd.read_csv('../Data/y_test.csv', sep=",")

In [24]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)   


(569, 25) (569, 2)
(143, 25) (143, 2)


In [25]:
print(X_train.head())
print(y_train.head())

                              name  \
0  MF01Q4BC2_A10_047.fsa_channel_1   
1  MF01Q4BC2_A10_047.fsa_channel_2   
2  MF01Q4BC2_A10_047.fsa_channel_4   
3  MF01Q4BC3_A12_048.fsa_channel_1   
4  MF01Q4BC3_A12_048.fsa_channel_2   

   value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8  \
0                                      510679.764022                  
1                                      496638.113174                  
2                                      463396.574771                  
3                                      554854.401846                  
4                                      420999.836988                  

   value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8  \
0                                      518815.590392                   
1                                      514965.032185                   
2                                      478174.054173                   
3                                      565875.131072    

In [55]:
# sort X_train and y_train rows by name to ensure alignment
X_train = X_train.sort_values(by="name").reset_index(drop=True)
y_train = y_train.sort_values(by="name").reset_index(drop=True)
print(y_train.head())

                              name Maincluster
0  MF01Q4BC2_A10_047.fsa_channel_1          Q9
1  MF01Q4BC2_A10_047.fsa_channel_2          Q9
2  MF01Q4BC2_A10_047.fsa_channel_4          Q9
3  MF01Q4BC3_A12_048.fsa_channel_1          Q9
4  MF01Q4BC3_A12_048.fsa_channel_2          Q9


In [47]:
#number of cluster in y_train
print(y_train['Maincluster'].unique())

#number of elements in each cluster
print(y_train['Maincluster'].value_counts())

#number of elements per channel (in the name column of y_train)
print(y_train['name'].str.extract(r'channel_(\d+)')[0].value_counts())

['Q9' 'Q7' 'Q3' 'Q1' 'Q5' 'Q4' 'Q6' 'Q2' 'Q8']
Maincluster
Q9    71
Q4    71
Q5    70
Q6    69
Q7    66
Q2    62
Q8    60
Q1    56
Q3    44
Name: count, dtype: int64
0
4    151
3    146
2    141
1    131
Name: count, dtype: int64


In [48]:
#number of cluster in y_train
print(y_test['Maincluster'].unique())

#number of elements in each cluster
print(y_test['Maincluster'].value_counts())

#number of elements per channel (in the name column of y_train)
print(y_test['name'].str.extract(r'channel_(\d+)')[0].value_counts())

['Q9' 'Q7' 'Q3' 'Q1' 'Q5' 'Q6' 'Q4' 'Q2' 'Q8']
Maincluster
Q9    25
Q5    22
Q7    18
Q1    16
Q6    15
Q2    14
Q4    13
Q8    12
Q3     8
Name: count, dtype: int64
0
1    47
2    37
3    32
4    27
Name: count, dtype: int64


In [26]:
print(X_train.columns)

Index(['name',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.8',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.8',
       'value__fft_coefficient__attr_"imag"__coeff_96',
       'value__fft_coefficient__attr_"real"__coeff_97',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.0',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.0',
       'value__cid_ce__normalize_False',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.6',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.6',
       'value__fft_coefficient__attr_"imag"__coeff_98',
       'value__fft_coefficient__attr_"real"__coeff_95',
       'value__fft_coefficient__attr_"imag"__coeff_89',
       'value__fft_coefficient__attr_"imag"__coeff_94',
       'value__change_quantiles__f_agg_"var"__isabs_True__qh_1.0__ql_0.4',
       'value__change_quantiles__f_agg_"var"__isabs_False__qh_1.0__ql_0.4',
    

In [34]:
# Dig in the parameters that were selected by tsfresh
from tsfresh.feature_extraction import settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
fc_parameters = ComprehensiveFCParameters()
feature_columns = X_train.drop(columns=["name"])
selected_features = tsf.feature_extraction.settings.from_columns(feature_columns.columns)
print(selected_features)

{'value': {'change_quantiles': [{'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.8}, {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.8}, {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.0}, {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.0}, {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.6}, {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.6}, {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.4}, {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.4}, {'f_agg': 'var', 'isabs': True, 'qh': 1.0, 'ql': 0.2}, {'f_agg': 'var', 'isabs': False, 'qh': 1.0, 'ql': 0.2}], 'fft_coefficient': [{'attr': 'imag', 'coeff': 96}, {'attr': 'real', 'coeff': 97}, {'attr': 'imag', 'coeff': 98}, {'attr': 'real', 'coeff': 95}, {'attr': 'imag', 'coeff': 89}, {'attr': 'imag', 'coeff': 94}, {'attr': 'imag', 'coeff': 91}, {'attr': 'real', 'coeff': 90}, {'attr': 'angle', 'coeff': 99}, {'attr': 'real', 'coeff': 88}, {'attr': 'angle', 'coeff': 84}], 'cid_ce': [{'normalize': False}], 'lempel_ziv_co

In [57]:
X_train = X_train.drop(columns=["name"])
X_test = X_test.drop(columns=["name"])
y_train = y_train['Maincluster']
y_test = y_test['Maincluster']

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from sklearn import set_config
set_config(transform_output='pandas')

clf = RandomForestClassifier(n_estimators=10, random_state=42)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)


print("Accuracy:", accuracy_score(y_pred, y_test))


Accuracy: 0.13286713286713286
